In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [66]:
input = [signal.rstrip('\n') for signal in open('DataN.txt')]
data = []
for signal in input:
    data.append(signal)


In [92]:
def derivatise(signals):
    filterd_signals= np.zeros((len(signals)))
    filterd_signals[0:2] = signals[0:2]

    T = 1/256
    for s in range(2,len(signals)-2):
        op = -float(signals[s-1])
        op -= (2*(float(signals[s-2])))
        op += (2*(float(signals[s+2])))
        op += (float(signals[s+1]))
        op *= (1/(8*T))
        
        filterd_signals[s] += np.square(op)
    
    return filterd_signals

In [93]:
def smoothen(signals, N):
    smoothed_signals = np.zeros((len(signals)))
    smoothed_signals[0:N] = signals[0:N]
    for s in range(N, len(signals)):
        for n in range(N, -1, -1):
            smoothed_signals[s] += (signals[s-n])
        smoothed_signals[s] *= (1/N)
    return smoothed_signals